## 5 Diferenças entre AdaBoost e GBM (Gradient Boosting Machine)

---

### 1. Forma de corrigir erros
- **AdaBoost:** Dá mais peso para exemplos mal classificados, focando nos erros do modelo anterior.
- **GBM:** Minimiza uma função de perda via gradiente descendente, ajustando os resíduos dos modelos anteriores.

---

### 2. Tipo de modelos base
- **AdaBoost:** Geralmente usa árvores rasas (stumps) com profundidade 1.
- **GBM:** Usa árvores um pouco mais profundas, capturando padrões mais complexos.

---

### 3. Flexibilidade na função de perda
- **AdaBoost:** Projetado para classificação, com perda exponencial.
- **GBM:** Permite várias funções de perda (regressão, classificação, ranking), sendo mais flexível.

---

### 4. Peso dos exemplos e amostragem
- **AdaBoost:** Ajusta explicitamente os pesos dos exemplos a cada iteração.
- **GBM:** Trabalha ajustando os resíduos sem alterar diretamente os pesos dos exemplos.

---

### 5. Velocidade e complexidade
- **AdaBoost:** Mais simples e rápido, devido aos modelos base mais simples.
- **GBM:** Mais lento e poderoso, pois realiza otimização numérica e usa árvores mais complexas.

---


In [1]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.datasets import make_hastie_10_2

X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

clf = HistGradientBoostingClassifier(max_iter=100).fit(X_train, y_train)
clf.score(X_test, y_test)

0.8965

## 5 Hiperparâmetros Importantes no GBM e Por Que São Essenciais

---

### 1. `n_estimators`  
- **O que faz:** Número de árvores a serem treinadas sequencialmente.  
- **Por que é importante:** Controla a complexidade do modelo. Poucos estimadores podem causar underfitting; muitos podem causar overfitting.

---

### 2. `learning_rate` (taxa de aprendizado)  
- **O que faz:** Escala a contribuição de cada árvore adicionada.  
- **Por que é importante:** Regulariza o aprendizado. Taxas menores exigem mais árvores, mas ajudam a evitar overfitting.

---

### 3. `max_depth`  
- **O que faz:** Profundidade máxima de cada árvore.  
- **Por que é importante:** Controla o nível de complexidade e interação capturada por cada árvore. Profundidades muito altas podem causar overfitting.

---

### 4. `min_samples_split` / `min_child_weight`  
- **O que faz:** Número mínimo de amostras necessárias para dividir um nó.  
- **Por que é importante:** Impede que a árvore cresça demais com divisões baseadas em poucos dados, ajudando a evitar overfitting.

---

### 5. `subsample`  
- **O que faz:** Fração dos dados usados para treinar cada árvore (amostragem sem reposição).  
- **Por que é importante:** Introduz aleatoriedade, reduzindo correlação entre árvores e ajudando na generalização do modelo.

---

 Ajustar bem esses hiperparâmetros é fundamental para equilibrar a **capacidade de aprendizado** do modelo com sua **capacidade de generalização**.


In [2]:
from sklearn.datasets import load_iris
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score

# 1. Carregar dados
X, y = load_iris(return_X_y=True)

# 2. Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Definir o modelo GBM
gbm = GradientBoostingClassifier(random_state=42)

# 4. Definir grade de hiperparâmetros para testar
param_grid = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [2, 3, 4],
    'min_samples_split': [2, 5, 10],
    'subsample': [0.6, 0.8, 1.0]
}

# 5. Configurar GridSearchCV
grid_search = GridSearchCV(estimator=gbm, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# 6. Ajustar o GridSearch no conjunto de treino
grid_search.fit(X_train, y_train)

# 7. Mostrar melhores hiperparâmetros encontrados
print("Melhores hiperparâmetros:")
print(grid_search.best_params_)

# 8. Avaliar o melhor modelo no conjunto de teste
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Acurácia no conjunto de teste: {acc:.4f}")


Melhores hiperparâmetros:
{'learning_rate': 0.01, 'max_depth': 3, 'min_samples_split': 5, 'n_estimators': 50, 'subsample': 1.0}
Acurácia no conjunto de teste: 1.0000


## Diferenças entre GBM clássico e Stochastic Gradient Boosting (Friedman)

### GBM clássico
- Treina cada árvore sequencialmente usando **todo o conjunto de dados** para calcular os resíduos.
- Cada árvore tenta corrigir os erros da soma das anteriores, minimizando a função de perda.
- Pode ser eficiente, mas tende a gerar árvores correlacionadas (alta variância).

### Stochastic Gradient Boosting
- Introduz **aleatoriedade (stocasticidade)** no processo de treinamento.
- Em vez de usar todo o conjunto, a cada iteração treina cada árvore em uma **amostra aleatória (subamostra)** dos dados, sem reposição.
- Essa amostragem reduz a correlação entre árvores e ajuda a melhorar a generalização.
- Geralmente melhora a robustez do modelo e reduz overfitting.

---

## O que são processos estocásticos?

- São processos que envolvem **elementos de aleatoriedade** ou incerteza.
- Em aprendizado de máquina, introduzir estocasticidade significa **usar amostras aleatórias**, ruído ou outras variações probabilísticas para tornar o modelo menos determinístico.
- No caso do Stochastic GBM, a amostragem aleatória dos dados a cada iteração é um exemplo clássico de um processo estocástico.

---

### Por que a estocasticidade ajuda no GBM?

- Reduz a correlação entre as árvores do ensemble, aumentando a diversidade.
- Evita que o modelo ajuste excessivamente os dados de treino (overfitting).
- Pode melhorar a velocidade e a estabilidade do aprendizado.

---

### Resumo rápido:

| Aspecto                 | GBM Clássico           | Stochastic GBM                        |
|------------------------|-----------------------|-------------------------------------|
| Uso dos dados          | Usa todo o dataset     | Usa subamostra aleatória (ex: 50%)  |
| Correlação entre árvores | Alta                  | Menor, mais diversidade             |
| Overfitting            | Maior risco            | Menor risco                         |
| Estocasticidade        | Não                    | Sim                                |

---

In [3]:
from sklearn.datasets import load_iris
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Carregar dados
X, y = load_iris(return_X_y=True)

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Gradient Boosting clássico (subsample=1.0)
gbm_classic = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,
                                         max_depth=3, subsample=1.0, random_state=42)
gbm_classic.fit(X_train, y_train)
pred_classic = gbm_classic.predict(X_test)
acc_classic = accuracy_score(y_test, pred_classic)
print(f"Acurácia GBM clássico: {acc_classic:.4f}")

# Stochastic Gradient Boosting (subsample < 1.0, ex: 0.5)
gbm_stochastic = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,
                                            max_depth=3, subsample=0.5, random_state=42)
gbm_stochastic.fit(X_train, y_train)
pred_stochastic = gbm_stochastic.predict(X_test)
acc_stochastic = accuracy_score(y_test, pred_stochastic)
print(f"Acurácia Stochastic GBM: {acc_stochastic:.4f}")


Acurácia GBM clássico: 1.0000
Acurácia Stochastic GBM: 1.0000
